In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("..")

import nidn
import torch

In [ ]:
# Start with the default config
cfg = nidn.load_default_cfg()

## Uniform Titanium-Oxide Ground Truth

In [ ]:
# Titanium Oxide Uniform
# Set grid specifics
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = 1
cfg.N_freq = 50
cfg.TRCWA_L_grid = [[1.0,0.0],[0.0,1.0]]
cfg.TRCWA_NG = 11
cfg.TRCWA_PER_LAYER_THICKNESS = [1.0]
cfg.freq_distribution = "log"

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 2e-6
cfg.physical_wavelength_range[1] = 1e-5

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)
eps_grid[:,:,0,:] = layer_builder.build_uniform_layer("titanium_oxide")

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("cfg.target_reflectance_spectrum = [", end="")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("cfg.target_transmittance_spectrum = [",end="")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")

## Uniform 3-Layer Ground-truth 

In [ ]:
# Start with the default config
cfg = nidn.load_default_cfg()

# Set grid specifics
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = 3
cfg.N_freq = 32
cfg.TRCWA_L_grid = [[1.0,0.0],[0.0,1.0]]
cfg.TRCWA_NG = 11
cfg.TRCWA_PER_LAYER_THICKNESS = [1.0]
cfg.freq_distribution = "linear"

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 1e-5
cfg.physical_wavelength_range[1] = 4e-5

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)
eps_grid[:,:,0,:] = layer_builder.build_uniform_layer("titanium_oxide")
eps_grid[:,:,1,:] = layer_builder.build_uniform_layer("germanium")
eps_grid[:,:,2,:] = layer_builder.build_uniform_layer("tantalum_pentoxide")

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("cfg.target_reflectance_spectrum = [", end="")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("cfg.target_transmittance_spectrum = [",end="")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")

## Patterned 2-Layer Ground-truth 

In [ ]:
# Start with the default config
cfg = nidn.load_default_cfg()

# Set grid specifics
cfg.Nx = 9
cfg.Ny = 9
cfg.N_layers = 2
cfg.N_freq = 32
cfg.TRCWA_L_grid = [[1.0,0.0],[0.0,1.0]]
cfg.TRCWA_NG = 11
cfg.TRCWA_PER_LAYER_THICKNESS = [1.0]
cfg.freq_distribution = "linear"

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 1e-5
cfg.physical_wavelength_range[1] = 4e-5

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)
eps_grid[:,:,0,:] = layer_builder.build_squared_layer("titanium_oxide","germanium")
eps_grid[:,:,1,:] = layer_builder.build_squared_layer("germanium","titanium_oxide")

In [ ]:
nidn.plot_epsilon_grid(eps_grid,cfg)

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("cfg.target_reflectance_spectrum = [", end="")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("cfg.target_transmittance_spectrum = [",end="")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")

## DBR Ground Truth

In [ ]:
# Set grid specifics
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = 33
layers_per_part = (cfg.N_layers - 1) // 4
cfg.N_freq = 32
cfg.TRCWA_L_grid = [[1.,0.0],[0.0,1.]]
cfg.TRCWA_NG = 11

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = (1.475)*1e-6
cfg.physical_wavelength_range[1] = (1.525)*1e-6

# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

In [ ]:
# Init eps_grid
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)

layer_builder = nidn.LayerBuilder(cfg)

thicknesses = []

n_2 = 2.2
n_1 = 1.5
eps_1 = n_1**2
eps_2 = n_2**2

lam0 = 1.50
d_1 = lam0/(4*n_1)
d_2 = lam0/(4*n_2)
d_defect = lam0/(2*n_1)

# Top DBR
for i in range(layers_per_part):
    eps_grid[:,:,2*i,:] = eps_1
    thicknesses.append(d_1)
    eps_grid[:,:,2*i+1,:] = eps_2
    thicknesses.append(d_2)
    
# Defect 
eps_grid[:,:,layers_per_part*2,:] = eps_1
thicknesses.append(d_defect)

# Bottom DBR
for i in range(layers_per_part,layers_per_part*2):
    eps_grid[:,:,2*i+1,:] = eps_2
    thicknesses.append(d_2)
    eps_grid[:,:,2*i+2,:] = eps_1
    thicknesses.append(d_1)

thicknesses.reverse()
cfg.TRCWA_PER_LAYER_THICKNESS = thicknesses

In [ ]:
# Plot the abs values of the produced grid
nidn.plot_epsilon_grid(eps_grid,cfg)

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
nidn.plot_spectrum(cfg,R,T)

In [ ]:
print("cfg.target_reflectance_spectrum = [", end="")
[print(f"{r.item():.8f}",end=",") for r in R]
print("]")
print("cfg.target_transmittance_spectrum = [",end="")
[print(f"{t.item():.8f}",end=",") for t in T]
print("]")